### 1. 导包

In [1]:
import pandas as pd
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Quora Question Pairs
是一个二分类问题，旨在判断一对问题是否具有相同的含义。这项任务的目标是识别出在意义上相似或者相同的问题，即判断它们是否可以被视为同一个问题的不同表述。
#### 任务特点
1. 输入：每个样本包含两个问题（例如，"What is the best way to learn Python?" 和 "How can I effectively learn Python?"）。
2. 

### 2. 加载数据集

In [4]:
# 加载数据集
dataset = load_dataset('quora', split='train', trust_remote_code=True)

Generating train split: 100%|██████████| 404290/404290 [00:10<00:00, 39471.49 examples/s]


In [5]:
dataset

Dataset({
    features: ['questions', 'is_duplicate'],
    num_rows: 404290
})

In [6]:
dataset_df=pd.DataFrame(dataset)